## Apiux & SII: Analisis exploratorio de datos y depuracion de data sociedades


In [1]:
from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
#from pyspark import SparkContext, SparkConf
#import pyspark
import pandas as pd
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

24/04/16 03:04:38 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/16 03:04:38 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 03:04:40 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/1

## Carga de relaciones societarias y depuracion de data

Primero, veamos los valores null en participacion de capital y participacion de utilidades.

In [3]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from libsdf.jab_soc_2023_inom where PORCENTAJE_CAPITAL is null or PORCENTAJE_UTILIDADES IS NULL").show()

24/04/16 03:05:18 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 743378f6-90f3-47f4-a58b-b2296ddb6f36
24/04/16 03:05:20 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...
24/04/16 03:05:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:05:54 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:06:09 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:06:24 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



Vemos que no hay valores nulos en la tabla. A continuacion veamos cuantos duplicados existen en las columnas de interes:
RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, pues todos
los calculos lo haremos basados en esta columnas.

In [4]:
spark.sql("select * from libsdf.jab_soc_2023_inom where RUT_SOCIEDAD like 'dwElNqcCQQiFyI3ic%' ").show()

+--------------------+--------------------+------------------+---------------------+------------------------+------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+--------------------+--------------------+------------------+---------------------+------------------------+------+
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0|                  202300|  RIAC|
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|  

In [5]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, count(*) as c from libsdf.jab_soc_2023_inom  group by  RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES order by c desc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from sociedad").createOrReplaceTempView("sociedad")

+--------------------+--------------------+------------------+---------------------+---+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|  c|
+--------------------+--------------------+------------------+---------------------+---+
|dwElNqcCQQiFyI3ic...|dwElNqcCQQiFyI3ic...|             100.0|                100.0| 12|
|E81vZtfA4Ybc4DMaf...|3BC/zRqDU+1GjN2Ks...|              99.0|                 99.0| 10|
|2AlrCTNxh4TadQyxC...|d4QdUJK8I1t8jAjfb...|              70.0|                 70.0| 10|
|2AlrCTNxh4TadQyxC...|I/YkZE/cZA9DBuO7E...|              30.0|                 30.0| 10|
|E81vZtfA4Ybc4DMaf...|8pRJYqtj522N9P1N8...|               1.0|                  1.0| 10|
|tC5mxZiKq7cNSWyNF...|LIkmP/LOUl9ME922f...|             99.99|                99.99| 10|
|tC5mxZiKq7cNSWyNF...|C4poSpbJFI6kSghEq...|              0.01|                 0.01| 10|
|J0MTHUaWHVxIuRb23...|4LWmtLgt7WF7AoW15...|             35.03|                35.03|  9|
|J0MTHUaWHVxIuRb23...

Donde seleccionamos los valores no repetidos. Haciendo nuevamente un recuento de los valores unicos,

In [6]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, count(*) as count from sociedad group by RUT_SOCIEDAD, RUT_SOCIO order by count desc ").show()

+--------------------+--------------------+-----+
|        RUT_SOCIEDAD|           RUT_SOCIO|count|
+--------------------+--------------------+-----+
|oKxjQk0RneBfnkJ/N...|wOXQ+k23eHQulvyO6...|    1|
|+Yqz4i3jUnOc+Jl00...|gxEFuvihw7+3ozxOJ...|    1|
|10Z0/Jq2swcrY4472...|l7ffjTqd4KcLgliIj...|    1|
|Ojoav9YqFql+loms9...|9RPywDJOjNtOEqvVr...|    1|
|6hH07h3oQoC6hTv13...|m+o27QNp8109CcsO3...|    1|
|GD7OKO0m7bWMe/Xo1...|97XD43WKaH8kQtN4K...|    1|
|vKngq8eQO6rrqw5HY...|H9CZYPMwinszXRe7s...|    1|
|4vDsxup0NJ1wtPo6W...|TnVvdSQqvcCFemR1l...|    1|
|+o05ofNn3t515IX1A...|t82P8u59Z05E7lV1a...|    1|
|OOzJifh2VIhW0eM6L...|+GD0F/bqbq+I8U3+s...|    1|
|MZWJbDAVfWa5Zx57R...|OAWOc1KUrFRKiu93F...|    1|
|qAJYUKeO36Txu3egi...|cYjgH7PtYnUZi617i...|    1|
|W7W5reshue3uDYTji...|E1BxDnhnaNJ/71uXQ...|    1|
|wTDUQa4Tr9Bfi2684...|LPjC+utYs3AsAYs2/...|    1|
|l91To4JDiXHqFf5h9...|QZwEeu9r5GCxGmjTJ...|    1|
|FpgLiP9CygcVs3inI...|7xE0ndsj4Oy0te5Dm...|    1|
|Tkk+dItPs0fykGj1/...|DrtEu9SVqW5d3BmHT...|    1|


Por lo visto tenemos unicidad de la relaciones sociedad socio, a juzgar por el recuento de las combinaciones. Ahora veremos cuanto suman los valores de PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES para cada una de las sociedades.


In [7]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL DESC").show()

+--------------------+-------+----------+
|        RUT_SOCIEDAD|CAPITAL|UTILIDADES|
+--------------------+-------+----------+
|E8eu9RvVl2VKYJaE7...|  110.0|     110.0|
|u82pDTR0SslxAp/ZT...|  110.0|     110.0|
|FNB44Z5TKNqHF4TYx...|  110.0|     100.0|
|cZDl+7O1w3LdSOizV...|  110.0|     110.0|
|VRCRk78lsO5weXzxa...|  110.0|     110.0|
|Lb2tQRQuCEvpm+2Jo...|  110.0|     110.0|
|i43JSRDNvap+tLHFu...|  110.0|     100.0|
|oR5s/MHfHD9ySi9yZ...|  110.0|     110.0|
|lRK9I77leupLzztR6...|  110.0|     110.0|
|4+UzWcxwl3hL0t+OC...|  110.0|     110.0|
|FhBSyWFAzWlnpx9xI...|  110.0|     110.0|
|QoXa9vGyA3qEbzRZV...|  110.0|     110.0|
|9Cb9dPdNp4ipEp1Ni...|  110.0|     110.0|
|tSeXL/llpGYXFbXII...|  110.0|     110.0|
|SB2LRoFxPwA9L0lSg...|  110.0|     110.0|
|MAeHdgJmh3+z4u+go...|  110.0|     110.0|
|ZRgBvcdLmrq6fdJpF...|  110.0|     110.0|
|djk+N5bNZhhXmuXv8...|  110.0|     110.0|
|9DP6CVeMzqGM5ngrT...|  110.0|     110.0|
|b8SQm9yLK8kcfiqCR...|  110.0|     110.0|
+--------------------+-------+----

In [8]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL ASC").show()

+--------------------+------------------+------------------+
|        RUT_SOCIEDAD|           CAPITAL|        UTILIDADES|
+--------------------+------------------+------------------+
|015/V4NMbTt4NqHqu...|11.289999999999841|11.289999999999841|
|88iZTIqYft+HO8Zrd...|              90.0|              90.0|
|ZRUGJaOStwYyzUZTB...|              90.0|              90.0|
|k+kPNCdFOaYbi11AS...|              90.0|              90.0|
|Oaa1jhxD5IlVJLTL0...|              90.0|              90.0|
|26gSjJQFEYeJdRgHR...|              90.0|              90.0|
|DBca1JG755RENOwum...|              90.0|              90.0|
|od2lKECOMlWXhlOMr...|              90.0|              90.0|
|1LlgPWRnIEbz7rbMj...|              90.0|              90.0|
|iXl2Ovj5D13sIGGuR...|              90.0|              90.0|
|+KDGQlV23t0xGZ8xt...|              90.0|              90.0|
|cQLLsIwdtzJfdIPU5...|              90.0|              90.0|
|CrCWavn+H87TDr3QL...|              90.0|              60.0|
|1dlFjOd2vfhLy1j1G...|  

In [9]:
anomalias_sociedades=spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD having CAPITAL>105 or CAPITAL<95").toPandas()
anomalias_sociedades.to_csv('/home/cdsw/data/processed/malla_societaria_procesada/anomalias_capital_sociedades.csv', index=False)
print(anomalias_sociedades.describe())

          CAPITAL  UTILIDADES
count  491.000000  491.000000
mean    94.785642   94.520611
std      8.458777    8.727403
min     11.290000   11.290000
25%     90.000000   90.000000
50%     91.000000   91.000000
75%     94.095000  100.000000
max    110.000000  127.000000


Ahora vamos las entradas con al menos un valor cero (que indica cero participacion porcentual)

In [10]:
spark.sql("select COUNT(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 OR PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
| 1057765|
+--------+



Por lo visto, tenemos 1057765 entradas donde al menos uno de ambos porcentajes es cero. Por otro lado, tenemos 
1053936 registros donde ambos porcentajes son cero.


Ahora veamos cuales porcentajes de capital son cero y luego los porcentajes de utilidades son cero.

In [11]:
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 and PORCENTAJE_UTILIDADES!=0").show()
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL!=0 and PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
|    2608|
+--------+



+--------+
|count(1)|
+--------+
|    1221|
+--------+



Para el analisis del problema de oscuridad, es mejor tener en cuenta los porcentajes de participacion de capital, porque los creditos se reparten segun la participacion societaria.
Ahora veamos cuantos tienen valores positivos mayores que 100 o negativos.

In [12]:
spark.sql("select * from sociedad WHERE PORCENTAJE_CAPITAL<0 or PORCENTAJE_CAPITAL>100 or PORCENTAJE_UTILIDADES<0 or PORCENTAJE_UTILIDADES>100").show()

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|qjkqLEYc5TRVKEMYf...|ifIlRQ1yYkDigN1Ii...|            100.01|                100.0|
+--------------------+--------------------+------------------+---------------------+



El cual es solo un valor levemente superior a 100 %. Seleccionamos los que no tienen valores cero en PORCENTAJE_CAPITAL.
IMPORTANTE: de ser utilizado PORCENTAJE_UTILIDADES se debe filtrar sobre esa columna.

In [13]:
spark.sql("select * from sociedad where PORCENTAJE_CAPITAL!=0").createOrReplaceTempView("sociedad")

In [14]:
spark.sql("select count(*) from sociedad where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()
spark.sql("select *  from libsdf.jab_soc_2023_inom where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()

+--------+
|count(1)|
+--------+
|    5695|
+--------+



+--------------------+--------------------+------------------+---------------------+------------------------+------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+--------------------+--------------------+------------------+---------------------+------------------------+------+
|Qbau/6SlJ/lEcKUDF...|RQ0TnfBpncysCFzap...|              0.03|                 0.03|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|i8odveNc3AEZRZWI7...|              0.01|                 0.01|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|9cDQjPbdfB1EfPUmw...|              0.02|                 0.02|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|7T3+u8inor8kIrUtm...|              0.02|                 0.02|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|CbAnOeVKkXKx8w0q3...|              0.01|                 0.01|                  202300| F1948|
|Qbau/6SlJ/lEcKUDF...|dEi5RAN/WiSIYuK1s...|              0.01|  

In [15]:
sociedad=spark.sql("select * from sociedad").toPandas()
sociedad.to_csv('/home/cdsw/data/processed/malla_societaria_procesada/sociedades_participacion_capital_nozero.csv', index=False)